In [ ]:
import os
from more_itertools import bucket
import pandas as pd
import re

In [ ]:
def js_to_python(s):
    d = {
        "true": "True",
        "false": "False",
        "null": "None",
        "[]": "None",
        "][": "],["
    }
    for k, v in d.items():
        s = s.replace(k, v)
    return s

def read_line(s):
    try:
        a, b = eval(js_to_python(s))
        return a
    except Exception as e:
        print(e)
        return eval(js_to_python(s))
        return []

def listdecorator(f):
    return lambda x: list(f(x))

@listdecorator
def get_trial_data(exp_dicts):
    return filter(is_trial, exp_dicts)
        
def is_trial(d):
    st = 'stimulus_type'
    return (d[st] == 'test') if (st in d.keys()) else False

def get_all_stimuli(trials):
    return [os.path.basename(trial[s]) for trial in trials for s in ['left', 'right']]

def get_group(fn):
    for g in ['low', 'high', 'medium']:
        if g in fn:
            return g

def nonempty_vp_idx(Ns, thr=0):
    return map(lambda x: x[0], (filter(lambda x: x[1] > thr, enumerate(Ns))))

In [ ]:
prefix = 'experiment'
result_files = {
    # fn = "../data/experiment/jatos_results_20230102094455.txt"  # single vp export
    # fn = "../data/experiment/jatos_results_20230102094507.txt"  # export all
    'pilot_general_multiple': "../data/experiment/jatos_results_20230110083507.txt",
    'pilot_personal_multiple': "../data/experiment/jatos_results_20230110100535.txt",
    'first_test_csv': "../data/experiment/jatos_results_20230111150257.txt",
    'second_pilot': "../data/experiment/jatos_results_20230116082513.txt",
    'experiment': "../data/experiment/jatos_results_20230120125730.txt",
    'experiment': "../data/experiment/jatos_results_20230124105303.txt"
}
fn = result_files[prefix]

# with open(fn) as fh:
#     raw_data = [read_line(s) for s in fh]
# len(raw_data)

df = pd.read_csv(fn)
raw_data = [list(filter(is_trial, df.to_dict('records')))]
len(raw_data[0])
# participants

participant_series = (df.groupby('sona').trial_index.agg(len) > 400)
participants = participant_series.index[participant_series]

In [ ]:
participant_series = (df.groupby('subject').trial_index.agg(len) > 400)
participants = participant_series.index[participant_series]
raw_data = [list(filter(is_trial, df[df.subject == p].to_dict('records'))) for p in participants]
len(raw_data)

In [ ]:
df = df[df.subject.isin(participants)]
df['time_elapsed'] = df['time_elapsed'].astype(int)

In [ ]:
exp_durations = ( df.groupby('subject').agg(lambda df: df.iloc[-1]).time_elapsed - df.groupby('subject').agg(lambda df: df.iloc[0]).time_elapsed ) / 60000
exp_durations.mean()

In [ ]:
# with open('check_pairs_exp.txt') as fh:
#     s = js_to_python(fh.read())

# trials = eval(s)
# raw_data = [trials]

# with open('check_pairs_exp.txt', 'w') as fh:
#     fh.write(s)

In [ ]:
Ns = [len(s) for s in raw_data]
print(Ns)

In [ ]:
def bias(trials):
    c = {g: 0 + 0j for g in ['low', 'medium', 'high']}
    for trial in trials:
        left, right = [os.path.basename(trial[lr]) for lr in ['left', 'right']]
        g = re.findall("low|high|medium", left)[0]
        if left == right:
            if trial['response'] == 'f':
                c[g] += 1
            else:
                c[g] += 1j
    return c

In [ ]:
6 * 6 *3 + 3 * 6 * 2

In [ ]:
3 * 6 * 2

In [ ]:
# len(trials)/3
7/49
240 * 7 / (7 * 7), 240 * 2 / (7 * 7 * 3)
# trials
(3 * (6 * 6 - 6 + 6 * 2)) + 3 * 3 * 2
(3 * (6 * 6 - 6 + 6 * 2)) + 3 * 3
(3 * (6 * 7))# + 3 * 3
# (3 * (6 * 6 - 6 + 6 * 2))

In [ ]:
groups = ['low', 'medium', 'high']

for vp in nonempty_vp_idx(Ns, 100):
    trials = get_trial_data(raw_data[vp])
    b = bias(trials)
    print(vp, b, sum(abs(v) for k, v in b.items()))
    stimulus_set = set(get_all_stimuli(trials))
    grouped_stimuli = {g: list(bucket(stimulus_set, lambda x: get_group(x))[g]) for g in ['low', 'medium', 'high']}
    losswin_counts = {}
    for g in ['low', 'high', 'medium']:
        losswin_counts[g] = {}
        for i in range(len(grouped_stimuli[g])):
            p1 = grouped_stimuli[g][i]
            for p2 in grouped_stimuli[g][(i+1):]:
                losswin_counts[g][(p1, p2)] = [0, 0]
            

    for trial in trials:
        left, right = [os.path.basename(trial[lr]) for lr in ['left', 'right']]
        if left == right:
            continue
        g = get_group(left)
        idx =  0 if trial['response'] == 'f' else 1
        try:
            losswin_counts[g][(left, right)][idx] += 1
        except KeyError:
            try:
                losswin_counts[g][(right, left)][idx-1] += 1
            except KeyError as e:
                print(e)

    for g in ['low', 'medium', 'high']:
        df = pd.DataFrame(losswin_counts[g]).T
        df = df.rename({0: "win1", 1: "win2"}, axis=1)
        pkl_fn = f"../data/processed/{prefix}_vp-{vp}_g-{g}.pkl"
        print('write:', pkl_fn)
        df.to_pickle(pkl_fn)

In [ ]:
df

In [ ]:
len(trials)
len(set([tuple([os.path.basename(trial[lr]) for lr in ['left', 'right']]) for trial in trials]))

In [ ]:
stimuli = []
classes = ['-100', '-66', '-33', '33', '66', '100'];

for g in groups:
    for i in range(6):
        
        stimuli.append((
            f"{g}_({classes[i]}).png",
            f'avgimg_wr_{g}.png',
        ))
        stimuli.append((
            f'avgimg_wr_{g}.png',
            f"{g}_({classes[i]}).png",
        ))
        
        for j in range(6):
            if i == j:
                continue
            stimuli.append( (f"{g}_({classes[i]}).png", f"{g}_({classes[j]}).png"))
            
for m in groups:
    for n in groups:
        stimuli.append((
            f'avgimg_wr_{m}.png',
            f'avgimg_wr_{n}.png',
        ))
        stimuli.append((
            f'avgimg_wr_{n}.png',
            f'avgimg_wr_{m}.png',
        ))
        
len(set(stimuli))# - set([tuple([os.path.basename(trial[lr]) for lr in ['left', 'right']]) for trial in trials])

In [ ]:
losswin_counts = {}
for g in ['low', 'high', 'medium']:
    losswin_counts[g] = {}
    for i in range(len(grouped_stimuli[g])):
        p1 = grouped_stimuli[g][i]
        for p2 in grouped_stimuli[g][(i+1):]:
            losswin_counts[g][(p1, p2)] = [0, 0]
            
for vp in nonempty_vp_idx(Ns, 100):
    trials = get_trial_data(raw_data[vp])
    b = bias(trials)
    print(vp, b, sum(abs(v) for k, v in b.items()))
    stimulus_set = set(get_all_stimuli(trials))
    grouped_stimuli = {g: list(bucket(stimulus_set, lambda x: get_group(x))[g]) for g in ['low', 'medium', 'high']}

    for trial in trials:
        left, right = [os.path.basename(trial[lr]) for lr in ['left', 'right']]
        if left == right:
            continue
        g = get_group(left)
        idx =  0 if trial['response'] == 'f' else 1
        try:
            losswin_counts[g][(left, right)][idx] += 1
        except KeyError:
            try:
                losswin_counts[g][(right, left)][idx-1] += 1
            except KeyError as e:
                print(e)
            #losswin_counts[g][(right, left)][idx-1] += 1

for g in ['low', 'medium', 'high']:
    df = pd.DataFrame(losswin_counts[g]).T
    df = df.rename({0: "win1", 1: "win2"}, axis=1)
    pkl_fn = f"../data/processed/pooled_{prefix}_g-{g}.pkl"
    df.to_pickle(pkl_fn)

In [ ]:
df
#f"../data/processed/pooled_{prefix}_vp-{vp}_g-{g}.pkl"